In [ ]:
# !pip install openai==0.28

In [ ]:
!pip install backoff

In [ ]:
!pip install openai

In [ ]:
import pandas as pd
import backoff #bilioteca para reexecutar o codigo caso ele caia em um erro determinado por nós, considerando que esse erro geralmente está relacioando a limite de taxa da API ou algo parecido
#ele espera um tempo para reexecutar
import os
from openai import OpenAI
import time
import numpy as np
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.metrics import (
    f1_score,
    precision_score,
    recall_score,
    classification_report
)

from sklearn.metrics import accuracy_score

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
sample_comments_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/IC/Teste_modelos/resultado_consolidado.csv')

In [ ]:
client = OpenAI(api_key="")

## Loading comments

In [ ]:
import re
import html
def pre_processamento (x):

  retorno = re.sub(r'http(s)?://\S+|www\.\S+', '', x)
  verdadeiro = re.search(r'&\S+', retorno)

  if verdadeiro:
        retorno = html.unescape(retorno)

  return retorno

In [ ]:
sample_comments_df.loc[:, 'Comentário'] = sample_comments_df.loc[:, 'Comentário'].apply(lambda x: pre_processamento(x))

In [ ]:
sample_comments_df.head()

,Unnamed: 0,id,Comentário,class_result,total_agreement,gpt_prompt1
0,0,htfz11y,Outra sugestão: procure o perfil no Instagram ...,2,1,1
1,1,hub93yq,Campeonato de quarto,1,1,1
2,2,hv1xsjp,Não entendi,1,1,1
3,3,hvsbpyg,"Mano, o Lucas Lima não esperou nem 10 minutos ...",0,1,0
4,4,hwzr3zr,Eu acho que ouvi um 🐄muuul,2,0,1


In [ ]:
sample_comments_df.shape

(171, 6)

In [ ]:
comment_ids = sample_comments_df['id'].tolist()

In [ ]:
comments_list = sample_comments_df['Comentário'].tolist()

## GPT-4o ft

In [ ]:
DEFAULT_TEMPERATURE = 0 #quanto menor, maior será a precisão e menor a criatividade
DEFAULT_MAX_TOKENS = 800
DEFAULT_MODEL = "gpt-4o-2024-08-06"

In [ ]:
DELIMITER = "###"

In [ ]:
INSTRUCTIONS = f""" Você é um assistente que determina a probabilidade de comentários do Reddit em
          Português do Brasil (PT-BR) de pertencerem a um dos sentimentos "Positivo", "Negativo" ou "Neutro".
          Você irá receber o texto de um comentário e a sua tarefa é determinar a probabilidade de predominar no texto um dos sentimentos especificados. Use as regras abaixo para realizar essa tarefa,
          seguindo especificamente o que elas determinam.
          1. Quanto mais próximo de -1 um comentário for, mais próximo do sentimento "Negativo" ele estará.
          2. Quanto mais próximo de 0 um comentário for, mais próximo do sentimento "Neutro" ele estará.
          3. Quanto mais próximo de 1 um comentário for, mais próximo do sentimento "Positivo" ele estará.
          5. Para cada comentário limite-se a ficar nessa faixa de valores, sem acrescentar texto explicativo, ultrapassar essa faixa de valores ou escolher outros sentimentos que não sejam os já
          definidos, sendo eles o "Positivo", "Negativo" e "Neutro".
""".strip()

In [ ]:
# @backoff.on_exception(backoff.expo, openai.error.RateLimitError)
def send_prompt(prompt: str, model: str = DEFAULT_MODEL) -> str:

    messages = [
        {"role": "system", "content": INSTRUCTIONS},
        {"role": "user", "content": prompt}
    ]

    resp = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=DEFAULT_TEMPERATURE,
        max_tokens=DEFAULT_MAX_TOKENS
    )

    answer = resp.choices[0].message.content.strip()

    return answer

In [ ]:
predictions_sample = []

In [ ]:
processed_ids = {}

In [ ]:
len(processed_ids)

0

In [ ]:
%%time

c = 1
for c_id, comment in zip(comment_ids, comments_list):
    if c_id in set(processed_ids):
        continue
    prompt = f"Comentário: {comment}"

    answer = send_prompt(prompt=prompt, model='gpt-4o-2024-08-06')

    if c % 10 == 0 :
        print(f"collected predictions for {c} comments...")
        time.sleep(20)

    predictions_sample.append({
        'id': c_id,
        'text': comment,
        'prediction': answer
    })

    c += 1

collected predictions for 10 comments...
collected predictions for 20 comments...
collected predictions for 30 comments...
collected predictions for 40 comments...
collected predictions for 50 comments...
collected predictions for 60 comments...
collected predictions for 70 comments...
collected predictions for 80 comments...
collected predictions for 90 comments...
collected predictions for 100 comments...
collected predictions for 110 comments...
collected predictions for 120 comments...
collected predictions for 130 comments...
collected predictions for 140 comments...
collected predictions for 150 comments...
collected predictions for 160 comments...
collected predictions for 170 comments...
CPU times: user 3.91 s, sys: 350 ms, total: 4.26 s
Wall time: 7min 22s


##Analise do dados

In [ ]:
predictions_df = pd.DataFrame(predictions_sample)

In [ ]:
processed_ids = predictions_df['id'].unique().tolist()

In [ ]:
predictions_df.head()

,id,text,prediction
0,htfz11y,Outra sugestão: procure o perfil no Instagram ...,0.2
1,hub93yq,Campeonato de quarto,0.1
2,hv1xsjp,Não entendi,0.0
3,hvsbpyg,"Mano, o Lucas Lima não esperou nem 10 minutos ...",-0.6
4,hwzr3zr,Eu acho que ouvi um 🐄muuul,0.1


In [ ]:
predictions_df.shape

(171, 3)

In [ ]:
def rotulacao(prob):

  if prob == 'Probabilidade de sentimento: -0.2':
    return 0

  prob = float(prob)

  if prob <= -0.2:
    return 0
  elif prob >= 0.2:
    return 2
  else:
    return 1


In [ ]:
predictions_df['prediction'].value_counts()

,count
prediction,
0.1,37
-0.8,34
-0.7,25
0.8,13
-0.6,12
0.2,11
-0.3,10
0.0,6
-0.9,5


In [ ]:
float(predictions_df.loc[:, 'prediction'][0]), predictions_df.loc[:, 'prediction'][0]

(0.2, '0.2')

In [ ]:
predictions_df.loc[:, 'prediction'] = predictions_df.loc[:,'prediction'].apply(lambda x: rotulacao(x))


In [ ]:
predictions_df.loc[:, 'prediction']

,prediction
0,2
1,1
2,1
3,0
4,1
...,...
166,1
167,2
168,1
169,2


In [ ]:
len(predictions_df[predictions_df['prediction'] < -1]), len( predictions_df[predictions_df['prediction'] > 1])

(0, 34)

In [ ]:
merge = pd.merge(sample_comments_df, predictions_df, how='inner', on='id')
merge.rename(columns={"prediction": "gpt_prompt2"}, inplace=True)

In [ ]:
merge.drop('text', inplace=True, axis=1)
merge

,Unnamed: 0,id,Comentário,class_result,total_agreement,gpt_prompt1,gpt_prompt2
0,0,htfz11y,Outra sugestão: procure o perfil no Instagram ...,2,1,1,2
1,1,hub93yq,Campeonato de quarto,1,1,1,1
2,2,hv1xsjp,Não entendi,1,1,1,1
3,3,hvsbpyg,"Mano, o Lucas Lima não esperou nem 10 minutos ...",0,1,0,0
4,4,hwzr3zr,Eu acho que ouvi um 🐄muuul,2,0,1,1
...,...,...,...,...,...,...,...
166,166,iwyyax0,1 O que fizeram em Los Angeles foi outra cois...,0,0,1,1
167,167,iyvd0es,"A última que eu tomei foi no meio desse ano, e...",2,1,2,2
168,168,izqc5yj,Lógico que desabou kkkkk,1,0,1,1
169,169,j0kxoll,como é bom ter democracia,2,1,2,2


In [ ]:
from sklearn.metrics import (
    f1_score,
    precision_score,
    recall_score,
    classification_report
)
from sklearn.metrics import accuracy_score

In [ ]:
accuracy_score(list(merge.loc[:, "class_result"]), list(merge.loc[:, "gpt_prompt2"]))

0.7894736842105263

In [ ]:
f1_score(list(merge.loc[:, "class_result"]), list(merge.loc[:, "gpt_prompt2"]), average='weighted')

0.7882957135007633

In [ ]:
precision_score(list(merge.loc[:, "class_result"]), list(merge.loc[:, "gpt_prompt2"]), average='weighted')

0.7880250149488164

In [ ]:
recall_score(list(merge.loc[:, "class_result"]), list(merge.loc[:, "gpt_prompt2"]), average='weighted')

0.7894736842105263

In [ ]:
merge

,Unnamed: 0,id,Comentário,class_result,total_agreement,gpt_prompt1,gpt_prompt2
0,0,htfz11y,Outra sugestão: procure o perfil no Instagram ...,2,1,1,2
1,1,hub93yq,Campeonato de quarto,1,1,1,1
2,2,hv1xsjp,Não entendi,1,1,1,1
3,3,hvsbpyg,"Mano, o Lucas Lima não esperou nem 10 minutos ...",0,1,0,0
4,4,hwzr3zr,Eu acho que ouvi um 🐄muuul,2,0,1,1
...,...,...,...,...,...,...,...
166,166,iwyyax0,1 O que fizeram em Los Angeles foi outra cois...,0,0,1,1
167,167,iyvd0es,"A última que eu tomei foi no meio desse ano, e...",2,1,2,2
168,168,izqc5yj,Lógico que desabou kkkkk,1,0,1,1
169,169,j0kxoll,como é bom ter democracia,2,1,2,2


In [ ]:
merge.to_csv('/content/drive/MyDrive/Colab Notebooks/IC/Teste_modelos/resultado_consolidado.csv')